# 📌 Laboratorio: Application Load Balancer con Cloud Armor

## 🌐 Introducción General
El **Balanceo de Carga de Aplicaciones** en **Google Cloud** opera en el borde de la red global de Google, en los **Puntos de Presencia (POP)** distribuidos mundialmente. Cuando un usuario accede a un servicio detrás de un **Application Load Balancer**, el tráfico ingresa al POP más cercano y se distribuye al backend con capacidad disponible.

🛡️ **Cloud Armor** permite crear listas de IPs permitidas (**allowlist**) o bloqueadas (**denylist**) en el borde de la red, restringiendo el acceso al balanceador desde fuentes maliciosas, protegiendo recursos y optimizando la seguridad.

💡 **Analogía contable**: Un **Application Load Balancer** es como un sistema centralizado que distribuye transacciones financieras a sucursales contables según su capacidad. **Cloud Armor** actúa como un filtro que bloquea accesos no autorizados a los registros financieros.

## 🧪 Actividades del Laboratorio
Configurarás un **Application Load Balancer** con backends globales, realizarás una prueba de estrés y bloquearás una IP con **Cloud Armor**.

## 🎯 Objetivos
- Crear reglas de firewall para tráfico HTTP y **health checks**.
- Configurar dos **plantillas de instancias**.
- Crear dos **grupos de instancias gestionados**.
- Configurar un **Application Load Balancer** con soporte IPv4 e IPv6.
- Realizar una prueba de carga.
- Denegar una IP con **Cloud Armor**.

💡 **Contexto empresarial**: Similar a configurar un ERP para distribuir transacciones financieras entre sucursales y proteger datos sensibles con políticas de acceso.

Para más información, consulta la [Documentación de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs) y la [Documentación de Cloud Armor](https://cloud.google.com/armor/docs).

## ⚙️ Requisitos Previos y Configuración

### Antes de Comenzar
- Lee las instrucciones completas. El tiempo del laboratorio no se puede pausar.
- Usa credenciales temporales proporcionadas por el laboratorio.
- Usa **Google Chrome** en modo **Incógnito** para evitar conflictos.
- **No uses cuentas personales** para evitar cargos.

### Instrucciones para Iniciar el Laboratorio
1. Haz clic en **Start Lab**.
2. Copia las credenciales temporales (usuario y contraseña).
3. Abre la **Consola de Google Cloud** en modo incógnito.
4. Inicia sesión con las credenciales.
5. Acepta los términos, sin configurar recuperación ni pruebas gratuitas.

💡 **Contexto empresarial**: Como preparar una auditoría contable con credenciales temporales para acceder a un sistema ERP.

### Navegación en Google Cloud Console
- Usa el **menú de navegación (☰)** o la barra de búsqueda para acceder a servicios.

| Término Técnico | Explicación Contable |
|-----------------|---------------------|
| **Balanceador de carga** | Repartidor de tareas que distribuye transacciones a servidores disponibles. |
| **Punto de presencia (POP)** | Sucursal de Google que recibe tráfico de usuarios. |
| **Backend** | Equipo contable que procesa solicitudes. |
| **Cloud Armor** | Filtro de seguridad que bloquea accesos no autorizados. |
| **Allowlist / Denylist** | Listas que definen IPs confiables o no confiables. |
| **Plantilla de instancia** | Modelo para crear servidores contables idénticos. |
| **Grupo de instancias gestionado** | Unidad de producción que se gestiona automáticamente. |
| **IPv4 / IPv6** | Identificadores de dispositivos en internet. |
| **Prueba de estrés** | Test para evaluar la capacidad del sistema bajo alta carga. |

## 🛠️ Tarea 1: Configurar Reglas de Firewall para HTTP y Health Checks

### Crear la Regla de Firewall para Tráfico HTTP
1. Navega a **VPC network > Firewall**.
2. Observa las reglas predeterminadas: `default-allow-icmp`, `default-allow-internal`, `default-allow-rdp`, `default-allow-ssh`.
3. Haz clic en **Create Firewall Rule**.
4. Configura:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `default-allow-http` |
| **Network** | `default` |
| **Targets** | Specified target tags |
| **Target tags** | `http-server` |
| **Source filter** | IPv4 Ranges |
| **Source IPv4 ranges** | `0.0.0.0/0` |
| **Protocols and ports** | TCP: `80` |

5. Haz clic en **Create**.

**Comando equivalente**:
```bash
gcloud compute firewall-rules create default-allow-http --network=default --action=ALLOW --rules=tcp:80 --source-ranges=0.0.0.0/0 --target-tags=http-server
```
**Explicación**:
- Permite tráfico HTTP (puerto 80) desde cualquier IP (`0.0.0.0/0`) a instancias con la etiqueta `http-server`.

💡 **Analogía contable**: Como permitir que auditores externos accedan a reportes financieros públicos.

### Crear la Regla de Firewall para Health Checks

1. Haz clic en **Create Firewall Rule**.

2. Configura:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `default-allow-health-check` |
| **Network** | `default` |
| **Targets** | Specified target tags |
| **Target tags** | `http-server` |
| **Source filter** | IPv4 Ranges |
| **Source IPv4 ranges** | `130.211.0.0/22`, `35.191.0.0/16` |
| **Protocols and ports** | TCP |

3. Haz clic en **Create**.

**Comando equivalente**:
```bash
gcloud compute firewall-rules create default-allow-health-check --network=default --action=ALLOW --rules=tcp --source-ranges=130.211.0.0/22,35.191.0.0/16 --target-tags=http-server
```
**Explicación**:
- Permite tráfico TCP desde las IPs de **health checks** de Google Cloud a instancias con la etiqueta `http-server`.
- **Nota**: Separa los rangos de IP con una coma sin espacios.

💡 **Contexto empresarial**: Como autorizar al equipo de auditoría interna para verificar la salud financiera de las sucursales.

| Tarea | Descripción |
|-------|-------------|
| **Regla default-allow-http** | Permite tráfico HTTP (puerto 80) desde cualquier IP. |
| **Regla default-allow-health-check** | Permite tráfico desde IPs de health checks. |
| **Etiqueta usada** | `http-server` |
| **Red utilizada** | `default` |
| **Protocolo** | TCP |
| **Rango de IPs (HTTP)** | `0.0.0.0/0` |
| **Rangos IP (health checks)** | `130.211.0.0/22`, `35.191.0.0/16` |

## 🧩 Tarea 2: Configurar Plantillas de Instancia y Grupos de Instancias

### Configurar Plantillas de Instancia

1. Navega a **Compute Engine > Instance templates**.
2. Haz clic en **Create instance template**.
3. Configura **Region 1-template**:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `Region 1-template` |
| **Location** | Global |
| **Series** | E2 |
| **Machine Type** | `e2-micro` |
| **Network tags** | `http-server` |
| **Network** | `default` |
| **Subnetwork** | `default` (Region 1) |
| **Startup script** | `gs://cloud-training/gcpnet/httplb/startup.sh` |

4. Haz clic en **Create**.

5. Crea **Region 2-template** (clic en **CREATE SIMILAR** desde `Region 1-template`):

| Propiedad | Valor |
|-----------|-------|
| **Name** | `Region 2-template` |
| **Subnetwork** | `default` (Region 2) |
| **Network tags** | `http-server` |

6. Haz clic en **Create**.

**Explicación**:
- El script `startup.sh` instala **Apache** y muestra la IP del cliente, nombre de la VM, región y zona.
- **Nota**: Reemplaza `Region 1` y `Region 2` con las regiones asignadas (ej. `us-central1`, `us-east1`).

💡 **Analogía contable**: Como crear plantillas para sucursales contables con configuraciones estándar.

### Crear Grupos de Instancias Gestionados

1. Navega a **Compute Engine > Instance groups**.

2. Crea **Region 1-mig**:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `Region 1-mig` |
| **Instance template** | `Region 1-template` |
| **Location** | Multiple zones |
| **Region** | `Region 1` |
| **Minimum number of instances** | 1 |
| **Maximum number of instances** | 2 |
| **Autoscaling signals** | CPU utilization |
| **Target CPU utilization** | 80 |
| **Initialization period** | 45 |

3. Haz clic en **Create**.

4. Crea **Region 2-mig**:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `Region 2-mig` |
| **Instance template** | `Region 2-template` |
| **Region** | `Region 2` |

💡 **Contexto empresarial**: Como establecer equipos contables que se escalan automáticamente según la carga de transacciones.

### Verificar los Backends

1. Navega a **Compute Engine > VM instances**.

2. Accede a la IP externa de una VM de `Region 1-mig` y `Region 2-mig`.

3. Verifica que se muestre:
   - IP del cliente.
   - Nombre de la VM.
   - Ubicación del servidor.

| Elemento | Valor / Descripción |
|----------|---------------------|
| **Plantillas** | `Region 1-template`, `Region 2-template` |
| **Tipo de máquina** | `e2-micro` |
| **Script de inicio** | Instala Apache, muestra IP, región, zona |
| **Etiqueta de red** | `http-server` |
| **Subredes** | `default` (Region 1 y Region 2) |
| **Grupos creados** | `Region 1-mig`, `Region 2-mig` |
| **Escalado automático** | CPU > 80% |
| **Verificación** | Acceso HTTP exitoso |

## 🌐 Tarea 3: Configurar el Application Load Balancer

### Configuración del Balanceador

1. Navega a **Networking > Network Services > Load balancing**.
2. Haz clic en **Create load balancer** > **Application Load Balancer HTTP(S)** > **Next**.
3. Selecciona:
   - **Public facing (external)**
   - **Best for global workloads**
   - **Global external Application Load Balancer**
4. Haz clic en **Configure**.
5. Nombre: `http-lb`.

### Configurar el Frontend

1. En **Frontend configuration**, configura IPv4:

| Propiedad | Valor |
|-----------|-------|
| **Protocol** | HTTP |
| **IP version** | IPv4 |
| **IP address** | Ephemeral |
| **Port** | 80 |

2. Haz clic en **Done**.

3. Añade IPv6:

| Propiedad | Valor |
|-----------|-------|
| **Protocol** | HTTP |
| **IP version** | IPv6 |
| **IP address** | Auto-allocate |
| **Port** | 80 |

4. Haz clic en **Done**.

💡 **Contexto empresarial**: Como configurar un sistema ERP para aceptar transacciones desde diferentes protocolos.

### Configurar el Backend

1. En **Backend configuration**, crea el servicio `http-backend`:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `http-backend` |
| **Instance group** | `Region 1-mig` |
| **Port numbers** | 80 |
| **Balancing mode** | Rate |
| **Maximum RPS** | 50 |
| **Capacity** | 100 |

2. Añade otro backend:

| Propiedad | Valor |
|-----------|-------|
| **Instance group** | `Region 2-mig` |
| **Port numbers** | 80 |
| **Balancing mode** | Utilization |
| **Maximum backend utilization** | 80 |
| **Capacity** | 100 |

3. Crea un **health check**:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `http-health-check` |
| **Protocol** | TCP |
| **Port** | 80 |

4. Habilita **Enable Logging** (Sample Rate: 1).
5. Haz clic en **Create**.

💡 **Analogía contable**: Como asignar transacciones a sucursales según su capacidad, con auditorías periódicas para verificar su estado.

### Finalizar el Balanceador
1. Haz clic en **Review and finalize**.
2. Verifica la configuración.
3. Haz clic en **Create**.
4. Anota las IPs del balanceador: `[LB_IP_v4]`, `[LB_IP_v6]`.

| Elemento | Valor / Descripción |
|----------|---------------------|
| **Nombre** | `http-lb` |
| **Tipo** | Global, público, HTTP |
| **IP Frontend** | `[LB_IP_v4]`, `[LB_IP_v6]` |
| **Backends** | `http-backend` con `Region 1-mig`, `Region 2-mig` |
| **Modos de Balanceo** | Rate (50 RPS), Utilization (80% CPU) |
| **Health Check** | TCP, puerto 80 |
| **Logging** | Activado (Sample Rate 1) |

## ✅ Tarea 4: Probar el Application Load Balancer

### Acceder al Balanceador
1. Abre un navegador y accede a `http://[LB_IP_v4]`.
2. Si tu red soporta IPv6, prueba `http://[LB_IP_v6]`.
**Nota**: Puede demorar 5 minutos. Si ves errores 404/502, reintenta.

💡 **Contexto empresarial**: Como verificar que un sistema ERP distribuye transacciones correctamente.

### Prueba de Estrés
1. Navega a **Compute Engine > VM instances**.

2. Crea `siege-vm`:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `siege-vm` |
| **Region** | `Region 3` |
| **Zone** | `Zone 3` |
| **Series** | E2 |

3. Haz clic en **Create**.
4. En `siege-vm` (SSH), instala `siege`:
```bash
sudo apt-get -y install siege
```
**Explicación**: Instala la herramienta `siege` para simular carga.

5. Configura la IP del balanceador:
```bash
export LB_IP=[LB_IP_v4]
```
**Nota**: Reemplaza `[LB_IP_v4]` con la IP del balanceador.

6. Ejecuta la prueba de carga:
```bash
siege -c 150 -t120s http://$LB_IP
```
**Explicación**: Simula 150 conexiones simultáneas durante 120 segundos.

7. Monitorea en **Load balancing > Backends > http-backend > Monitoring**.
**Resultado**: El tráfico inicia en `Region 1-mig` y se distribuye a `Region 2-mig` si hay sobrecarga.

💡 **Analogía contable**: Como simular un pico de transacciones financieras y observar cómo se distribuyen entre sucursales.

### Denegar Acceso con Cloud Armor

1. Anota la IP externa de `siege-vm`: `[SIEGE_IP]`.
2. Navega a **Networking > Network Security > Cloud Armor policies**.
3. Haz clic en **Create policy**:

| Propiedad | Valor |
|-----------|-------|
| **Name** | `denylist-siege` |
| **Default rule** | Allow |
| **Match condition** | `[SIEGE_IP]` |
| **Action** | Deny |
| **Response code** | 403 |
| **Priority** | 1000 |
| **Target** | Backend service: `http-backend` |

4. Haz clic en **Create policy**.

5. Verifica el bloqueo desde `siege-vm`:
```bash
curl http://$LB_IP
```
**Resultado**: `403 Forbidden`.

6. Prueba nuevamente con `siege`:
```bash
siege -c 150 -t120s http://$LB_IP
```
**Resultado**: No muestra salida (bloqueado).

7. En el navegador, accede a `http://[LB_IP_v4]` (funciona, ya que la regla permite otras IPs).

💡 **Contexto empresarial**: Como bloquear el acceso de un usuario no autorizado a un sistema financiero.

### Ver Logs de Cloud Armor
1. Navega a **Network Security > Cloud Armor Policies > denylist-siege > Logs**.
2. Selecciona **Application Load Balancer > http-lb-forwarding-rule > http-lb**.
3. Ejecuta la consulta y verifica:
   - **httpRequest**: IP de `siege-vm`.
   - **jsonPayload**: `configuredAction: DENY`, política `denylist-siege`.

| Elemento | Valor / Explicación |
|----------|---------------------|
| **Propósito** | Verificar balanceo y control de acceso |
| **Herramienta** | `siege` en `siege-vm` |
| **Distribución** | Inicialmente `Region 1`, luego `Region 2` si hay sobrecarga |
| **Política Cloud Armor** | `denylist-siege` (bloquea `[SIEGE_IP]`) |
| **Verificación** | `403 Forbidden` desde `siege-vm`; acceso desde navegador |
| **Logs** | Confirman acción `DENY` para `[SIEGE_IP]` |

## 🚀 Conclusión

Este laboratorio te permitió:
1. Configurar reglas de firewall para HTTP y **health checks**.
2. Crear **plantillas de instancias** y **grupos gestionados**.
3. Implementar un **Application Load Balancer** global.
4. Realizar una prueba de carga con `siege`.
5. Proteger el sistema con **Cloud Armor**.

💡 **Beneficio empresarial**: Similar a configurar un ERP que distribuye transacciones financieras entre sucursales, protege datos sensibles y escala automáticamente según la demanda.

Para más información, consulta la [Documentación de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs) y la [Documentación de Cloud Armor](https://cloud.google.com/armor/docs).

**Nota sobre las imágenes**: [Insertar imágenes `image1.png` y `image2.png` aquí, que ilustran el esquema del balanceador de carga y las políticas de Cloud Armor.]